In [3]:
import json
import pandas as pd
from a2ginputstream import A2GClient
import random
import time
from datetime import datetime
from datetime import timedelta
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

## Read Data

In [4]:
USER_PAT, IKEY = "8f8ff571c7344ce9b6fae09f1ff9c5b1", "1c01a94bea5948faabda"
client = A2GClient(token=USER_PAT)
time_now = datetime.utcnow().replace(
    microsecond=0,
    second=0,
    minute=0
)

start_hour = time_now
end_hour = time_now + timedelta(hours=1)

movements_data:list[dict] = client.find(IKEY, {"start_on": {"$gte":start_hour}, "end_on":{"$lte":end_hour}}, delete_id=True)

2024-10-29 08:40:09,875 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443
2024-10-29 08:40:10,286 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/1c01a94bea5948faabda HTTP/11" 200 None
2024-10-29 08:40:10,294 - INFO - Getting inputstream 1c01a94bea5948faabda from A2G...
2024-10-29 08:40:10,298 - INFO - Downloading data from A2G...
2024-10-29 08:40:10,303 - DEBUG - Starting new HTTPS connection (1): v2streams.a2g.io:443
2024-10-29 08:40:10,897 - DEBUG - https://v2streams.a2g.io:443 "POST /QueryData/ExecutionPlanningFind HTTP/11" 200 119
2024-10-29 08:40:10,900 - INFO - The query stages are COLLSCAN
2024-10-29 08:40:10,900 - INFO - The index used in query is None
2024-10-29 08:40:10,901 - INFO - Total documents to download 31.
2024-10-29 08:40:10,902 - INFO - Batch 0/1
2024-10-29 08:40:10,904 - DEBUG - Starting new HTTPS connection (1): v2streams.a2g.io:443
2024-10-29 08:40:11,325 - DEBUG - https://v2streams.a2g.io:443 "POST /QueryData/FindAll?page=1&p

In [5]:
movements_data[0]

{'name': 'John Doe',
 'role': 'admin',
 'machines': 'C303',
 'status': 'inactive',
 'ton': 86,
 'start_on': datetime.datetime(2024, 10, 29, 11, 0, 5, 531000),
 'end_on': datetime.datetime(2024, 10, 29, 11, 5, 5, 532000),
 'awhen': datetime.datetime(2024, 10, 29, 11, 5, 6, 156000)}

## Consolidate data by user and machine

In [7]:
df_movements = pd.DataFrame(movements_data)
df_users = df_movements.groupby(by="name")[["ton"]].sum()
df_machines = df_movements.groupby(by="machines")[["ton"]].sum()

df_machines.reset_index(inplace=True)
df_users.reset_index(inplace=True)

df_users["time"] = time_now.isoformat()
df_machines["time"] = time_now.isoformat()

In [8]:
df_machines

,machines,ton,time
0,C301,428,2024-10-29T11:00:00
1,C302,808,2024-10-29T11:00:00
2,C303,332,2024-10-29T11:00:00


In [9]:
df_users

,name,ton,time
0,Ahbraham Smith,240,2024-10-29T11:00:00
1,Jane Doe,282,2024-10-29T11:00:00
2,John Doe,230,2024-10-29T11:00:00
3,John Smith,235,2024-10-29T11:00:00
4,Richard Alfredo,300,2024-10-29T11:00:00
5,Smith Alfred,281,2024-10-29T11:00:00


## Save in inputstream

In [10]:
IKEY_machine = "574dd3c216924478a03e"
client.insert_data(IKEY_machine, df_machines.to_dict(orient="records"))

IKEY_users = "0fa6688ad39f4622bdd9"
client.insert_data(IKEY_users, df_users.to_dict(orient="records"))

2024-10-29 08:40:49,363 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443


2024-10-29 08:40:50,754 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/574dd3c216924478a03e HTTP/11" 200 None
2024-10-29 08:40:50,766 - INFO - Getting inputstream 574dd3c216924478a03e from A2G...
2024-10-29 08:40:50,768 - WARNING - Inputstream discovered
2024-10-29 08:40:50,771 - DEBUG - Starting new HTTPS connection (1): v2streams.a2g.io:443
2024-10-29 08:40:51,965 - DEBUG - https://v2streams.a2g.io:443 "POST /Data/Insert HTTP/11" 200 None
2024-10-29 08:40:51,973 - INFO - batch 1, docs: [0 - 3] - 200 - {"message":"Data Received Success, DataToyMachine inputstream","data":null,"success":true,"errors":null}
2024-10-29 08:40:51,976 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443
2024-10-29 08:40:52,407 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/0fa6688ad39f4622bdd9 HTTP/11" 200 None
2024-10-29 08:40:52,419 - INFO - Getting inputstream 0fa6688ad39f4622bdd9 from A2G...
2024-10-29 08:40:52,421 - WARNING - Inputstream discovered
20